In [4]:
from uiautomator import device as d
d.info

{'currentPackageName': 'com.poupa.vinylmusicplayer',
 'displayHeight': 1794,
 'displayRotation': 0,
 'displaySizeDpX': 411,
 'displaySizeDpY': 731,
 'displayWidth': 1080,
 'productName': 'sdk_gphone_x86',
 'screenOn': True,
 'sdkInt': 28,
 'naturalOrientation': True}

In [3]:
d.screen.on()

In [8]:
d.press.camera()

True

In [14]:
d.orientation

'natural'

In [90]:
d.screenshot("home.png")

'home.png'

In [84]:
d.dump("hierarchy.xml", compressed=False)

'<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?>\r\n<hierarchy rotation="0">\r\n  <node index="0" text="" resource-id="com.android.systemui:id/navigation_bar_frame" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,1794][1080,1920]">\r\n    <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,1794][1080,1920]">\r\n      <node index="0" text="" resource-id="com.android.systemui:id/navigation_inflater" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" check

In [104]:
d(scrollable='true').info

{'bounds': {'bottom': 1794, 'left': 0, 'right': 1080, 'top': 63},
 'childCount': 7,
 'className': 'android.webkit.WebView',
 'contentDescription': None,
 'packageName': 'com.uberspot.a2048',
 'resourceName': None,
 'text': '2048',
 'visibleBounds': {'bottom': 1794, 'left': 0, 'right': 1080, 'top': 63},
 'checkable': False,
 'checked': False,
 'clickable': False,
 'enabled': True,
 'focusable': True,
 'focused': True,
 'longClickable': False,
 'scrollable': True,
 'selected': False}

In [82]:
d.dump("hierarchy.xml")

'<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?>\r\n<hierarchy rotation="1">\r\n  <node index="0" text="" resource-id="com.android.systemui:id/navigation_bar_frame" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[1794,0][1920,1080]">\r\n    <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[1794,0][1920,1080]">\r\n      <node index="0" text="" resource-id="com.android.systemui:id/menu" class="android.widget.ImageView" package="com.android.systemui" content-desc="Menu" checkable="false" checked="false" cl

In [40]:
d.watcher("AUTO_FC_WHEN_ANR").when(clickable="true").click(text="")

In [23]:
d(scrollable=True).fling.vert.backward()

False

In [21]:
d(clickable=True, instance=3).click()

True

In [88]:
import xml.sax
import os

def findFiles(rootdir):
    #find all files under the rootdir
    files = []
    list = os.listdir(rootdir)
    for subPath in list:
        path = os.path.join(rootdir,subPath)
        if os.path.isdir(path):
            files.extend(findFiles(path))
        if os.path.isfile(path):
            files.append(path)
    return files

# class resourceID():
#     def __init__(self, name, bounds):
#         self.name = name
#         self.bounds = bounds

class MyHandler( xml.sax.ContentHandler ):
    def __init__(self, resourceIDList):
        self.resourceIDList = resourceIDList
 
   # 元素开始事件处理
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        if attributes.get('package') != 'com.android.systemui':
            resourceID = str(attributes.get('resource-id'))
            bounds = str(attributes.get('bounds'))
            find = resourceID.find('/')
            if find != -1:
                resourceID = str(resourceID[find+1:])
                resourceIDList[resourceID] = bounds
                
 
    # 元素结束事件处理
#     def endElement(self, tag):
#         if self.CurrentData == "type":
#             print "Type:", self.type
#         self.CurrentData = ""
 
#    # 内容事件处理
#     def characters(self, content):
#         if self.CurrentData == "type":
#             self.type = content
#         elif self.CurrentData == "format":
#             self.format = content

class LayoutHandler (xml.sax.ContentHandler):
    def __init__(self, findList):
        self.findList = findList
 
   # 元素开始事件处理
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        if ('.' in tag) and (not 'android' in tag) and (not 'maxfour' in tag):
            resourceID = str(attributes.get('android:id'))
            find = resourceID.find('/')
            if find != -1:
                resourceID = resourceID[find+1:]
                if resourceID in self.findList.keys():
                    print(resourceID + " " + self.findList[resourceID] + " " + tag)
#         resourceID = str(attributes.get('android:id'))
#         find = resourceID.find('/')
#         if find != -1:
#             resourceID = resourceID[find+1:]
#             if resourceID in self.findList.keys():
#                 print(resourceID + " " + self.findList[resourceID] + " " + tag)

In [89]:
# 创建一个 XMLReader
parser = xml.sax.make_parser()
# turn off namepsaces
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
 
# 重写 ContextHandler
resourceIDList = {}
Handler = MyHandler(resourceIDList)
parser.setContentHandler( Handler )
parser.parse("hierarchy.xml")
files = findFiles('/Users/xikaioliver/Desktop/com.maxfour.music_8/resources/res/layout')
for file in files:
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    Handler = LayoutHandler(resourceIDList)
    parser.setContentHandler( Handler )
    parser.parse(file)

player_sliding_layout [0,0][1080,1794] com.sothree.slidinguppanel.SlidingUpPanelLayout
player_sliding_layout [0,0][1080,1794] com.sothree.slidinguppanel.SlidingUpPanelLayout
sliding_layout [0,0][1080,1794] com.sothree.slidinguppanel.SlidingUpPanelLayout
